In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from datetime import date, timedelta
from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots
from typing import List, Tuple
from statsmodels.tsa.seasonal import STL
from collections import defaultdict
from sklearn.metrics import SCORERS
from IPython.core.display import display, HTML
from scipy import stats

# Препроцессинг (Часть 1)

## Считываем данные 

In [2]:
df = pd.read_xml('electric power.xml')
df.rename(columns={'DATE': 'date', 'VALUE': 'value'}, inplace=True)
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M:%S.%f').dt.date
df = df[['date', 'value']]
df

,date,value
0,2009-01-01,243887.0
1,2009-01-01,243068.0
2,2009-01-01,242266.0
3,2009-01-01,242659.0
4,2009-01-01,243724.0
...,...,...
17513,2010-12-31,371835.0
17514,2010-12-31,370674.0
17515,2010-12-31,369646.0
17516,2010-12-31,365917.0


## Проверяем наличие nan 

In [3]:
df.isna().values.any()

False

## Укрупним данные

In [4]:
df = df.groupby('date', as_index=False).sum()
df.set_index(['date'], inplace=True)
df

,value
date,
2009-01-01,5830312.0
2009-01-02,5865441.0
2009-01-03,5949114.0
2009-01-04,6067779.0
2009-01-05,6115772.0
...,...
2010-12-27,6624696.0
2010-12-28,7543819.0
2010-12-29,8790105.0


## Визуализируем данные

In [5]:
fig = px.line(df, y='value')
fig.update_layout(xaxis_title='Дата', yaxis_title='Объем потребления',
                  title='Объём потребления электроэнергии в период с 2009 по 2010 год')
fig.show()

Unsupported

In [6]:
fig = px.line(df, y='value')
fig.update_layout(xaxis_title='Дата', yaxis_title='Объем потребления',
                  title='Объём потребления электроэнергии в период с 2009 по 2010 год')
fig.add_vrect(x0="2009-06-30", x1="2009-08-17", fillcolor="Red", opacity=0.5, line_width=0)
fig.add_vrect(x0="2009-01-01", x1="2009-06-30", fillcolor="Orange", opacity=0.5, line_width=0)
fig.show()

Unsupported

In [7]:
df = df[df.index >= date(2009, 8, 17)]

fig = px.line(df, y='value')
fig.update_layout(xaxis_title='Дата', yaxis_title='Объем потребления',
                  title='Объём потребления электроэнергии в период с 2009 по 2010 год')
fig.add_vrect(x0="2009-09-25", x1="2009-09-28", fillcolor="OrangeRed", opacity=0.5, line_width=0, layer="below")
fig.add_vrect(x0="2009-10-09", x1="2009-10-12", fillcolor="OrangeRed", opacity=0.5, line_width=0, layer="below")
fig.add_vrect(x0="2009-12-04", x1="2009-12-08", fillcolor="OrangeRed", opacity=0.5, line_width=0, layer="below")
fig.add_vrect(x0="2010-05-21", x1="2010-05-24", fillcolor="OrangeRed", opacity=0.5, line_width=0, layer="below")
fig.add_vrect(x0="2010-06-25", x1="2010-06-28", fillcolor="OrangeRed", opacity=0.5, line_width=0, layer="below")
fig.add_vrect(x0="2010-09-17", x1="2010-09-21", fillcolor="OrangeRed", opacity=0.5, line_width=0, layer="below")
fig.add_vrect(x0="2010-12-22", x1="2010-12-29", fillcolor="OrangeRed", opacity=0.5, line_width=0, layer="below")
fig.show()

Unsupported

In [8]:
def show_trend_and_season(period: int, period_name: str):
    stl = STL(df['value'], period=period)
    res = stl.fit()

    fig = make_subplots(cols=1, rows=2, shared_xaxes=True)
    fig.add_scatter(x=res.trend.index, y=res.trend, name='Тренд', row=1, col=1)
    fig.add_scatter(x=res.seasonal.index, y=res.seasonal, name='Сезонность', row=2, col=1)
    fig.update_xaxes(title='День')
    fig.update_yaxes(title='Значение')
    fig.update_layout(title=f'Период: {period_name}')
    fig.show()

show_trend_and_season(7, 'неделя')
show_trend_and_season(30, 'месяц')
show_trend_and_season(365, 'год')

Unsupported

# Препроцессинг (Часть 2)

In [9]:
def expand(df: pd.DataFrame, by_month: bool = True, by_weekday: bool = True, ignore_months=(12,),
           ignore_weekdays=(7,)) -> pd.DataFrame:
    df = df.copy()
    if by_month:
        for month_number in range(1, 13):
            if month_number in ignore_months:
                continue
            df[f'month_{month_number}'] = df.apply(lambda row: row.name.month == month_number, axis=1).astype(int)

    if by_weekday:
        for weekday_number in range(1, 8):
            if weekday_number in ignore_weekdays:
                continue
            df[f'weekday_{weekday_number}'] = df.apply(lambda row: row.name.isoweekday() == weekday_number,
                                                       axis=1).astype(int)

    return df


def remove_outliers(df: pd.DataFrame, predictions: np.array, dates: List[date]) -> pd.DataFrame:
    df = df.copy()

    for date in dates:
        predictions = pd.DataFrame(data=predictions, index=df.index, columns=['value'])
        df.at[date, 'value'] = predictions.loc[date, 'value']

    return df


## Избавляемся от выбросов

In [33]:
expanded_df = expand(df)
expanded_df['day'] = range(len(expanded_df))
expanded_df

,value,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,day
date,,,,,,,,,,,,,,,,,,,
2009-08-17,3398401.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2009-08-18,3505239.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
2009-08-19,3554215.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,2
2009-08-20,3770158.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,3
2009-08-21,3873099.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-27,6624696.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,497
2010-12-28,7543819.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,498
2010-12-29,8790105.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,499


In [11]:
model_all = LinearRegression()
x_columns = expanded_df.columns.to_list()
x_columns.remove('value')
model_all.fit(expanded_df[x_columns], expanded_df['value'])
predictions = model_all.predict(expanded_df[x_columns])

fig = go.Figure()
fig.add_scatter(x=expanded_df.index, y=predictions, name='Модель')
fig.add_scatter(x=df.index, y=df['value'], name='Исходные данные')
fig.update_layout(xaxis_title='Дата', yaxis_title='Объем потребления',
                  title='Объём потребления электроэнергии в период с 2009 по 2010 год')
fig.show()

Unsupported

In [12]:
df_without_outliers = remove_outliers(df, predictions,
                                      [date(2009, 9, 26), date(2009, 9, 27), date(2009, 10, 10), date(2009, 10, 11),
                                       date(2009, 12, 5), date(2009, 12, 6), date(2009, 12, 7), date(2010, 5, 22),
                                       date(2010, 5, 23), date(2010, 6, 26), date(2010, 6, 27), date(2010, 9, 18),
                                       date(2010, 9, 19), date(2010, 9, 20), date(2010, 12, 24), date(2010, 12, 25),
                                       date(2010, 12, 26), date(2010, 12, 27), date(2010, 12, 28)])

fig = go.Figure()
fig.add_scatter(x=df.index, y=df['value'], name='Исходные данные')
fig.add_scatter(x=df_without_outliers.index, y=df_without_outliers['value'], name='Без выбросов')
fig.update_layout(xaxis_title='Дата', yaxis_title='Объем потребления',
                  title='Объём потребления электроэнергии в период с 2009 по 2010 год')
fig.show()

Unsupported

# Обучение

In [13]:
def train_test_split(df: pd.DataFrame, number_of_test_days: int) -> Tuple[
    pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    max_day = df.index.max()

    x_columns = df.columns.to_list()
    x_columns.remove('value')

    train = df[df.index <= max_day - timedelta(days=number_of_test_days)]
    test = df[df.index > max_day - timedelta(days=number_of_test_days)]

    return train[x_columns], train['value'], test[x_columns], test['value']

In [14]:
metric_names = ['r2', 'max_error', 'neg_mean_absolute_error', 'neg_root_mean_squared_error']

In [15]:
def score_model(model: LinearRegression, X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame,
                y_test: pd.DataFrame) -> pd.DataFrame:
    scores_by_data = defaultdict(list)
    for metric_name in metric_names:
        scores_by_data['Train'].append(SCORERS[metric_name](model, X_train, y_train))
        if X_test is not None and y_test is not None:
            scores_by_data['Test'].append(SCORERS[metric_name](model, X_test, y_test))
            scores_by_data['All'].append(SCORERS[metric_name](model, pd.concat([X_train, X_test]), pd.concat([y_train, y_test])))
    return pd.DataFrame(scores_by_data, index=metric_names).T

In [16]:
def regression_coef(model, X, y):
    coef = pd.DataFrame(zip(['intercept'] + X.columns.tolist(), [model.intercept_] + model.coef_.tolist()),
                    columns=['predictor', 'coef'])
    
    X1 = np.append(np.ones((len(X),1)), X, axis=1)
    b = np.append(model.intercept_, model.coef_)
    MSE = np.sum((model.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
    var_b = MSE * (np.linalg.inv(np.dot(X1.T, X1)).diagonal())
    sd_b = np.sqrt(var_b)
    t = b / sd_b

    coef['pvalue'] = [2 * (1 - stats.t.cdf(np.abs(i), (len(X1) - 1))) for i in t]
    coef.set_index(['predictor'], inplace=True)
    coef.columns.name = coef.index.name
    coef.index.name = None

    return coef

In [17]:
def display_model_coefs(model: LinearRegression, X: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:
    features = model.feature_names_in_
    display(HTML(regression_coef(model, X[features], y).style.format({'coef': '{:.4f}', 'pvalue': '{:e}'}).to_html()))

In [18]:
def fit_model(df: pd.DataFrame, number_of_test_days: int) -> Tuple[LinearRegression, go.Figure, pd.DataFrame]:
    """
    Returns model, plot and scores
    """

    X_train, y_train, X_test, y_test = train_test_split(df, number_of_test_days)

    model = LinearRegression()
    model.fit(X_train, y_train)

    dates = pd.concat([X_train, X_test]).index
    predictions = model.predict(pd.concat([X_train, X_test]))

    fig = go.Figure()
    fig.add_scatter(x=dates, y=pd.concat([y_train, y_test]), name='Обучающая выборка')
    fig.add_scatter(x=X_test.index, y=y_test, name='Тестовая выборка', mode='lines')
    fig.add_scatter(x=dates, y=predictions, name='Модель')
    fig.update_layout(xaxis_title='Дата', yaxis_title='Объем потребления',
                  title='Объём потребления электроэнергии в период с 2009 по 2010 год')

    scores = score_model(model, X_train, y_train, X_test, y_test)

    display_model_coefs(model, X_train, y_train)

    return model, fig, scores

## Все колонки

In [19]:
expanded_df_without_outliers = expand(df_without_outliers)
expanded_df_without_outliers['day'] = range(len(expanded_df_without_outliers))

model_all, plot_all, scores_all = fit_model(expanded_df_without_outliers, 14)

predictor,coef,pvalue
intercept,6857634.1270,0.000000e+00
month_1,202804.2444,3.914858e-02
month_2,434003.5963,1.943250e-05
month_3,-238304.0708,1.458206e-02
month_4,-991224.0518,0.000000e+00
month_5,-2345920.7727,0.000000e+00
month_6,-3551904.3844,0.000000e+00
month_7,-4210186.2511,0.000000e+00
month_8,-3675261.8400,0.000000e+00
month_9,-2755093.0759,0.000000e+00


In [20]:
plot_all.show()

Unsupported

In [21]:
scores_all.style.highlight_max()

,r2,max_error,neg_mean_absolute_error,neg_root_mean_squared_error
Train,0.929792,-1380306.167733,-320867.126409,-413270.369546
Test,0.502677,-649318.640957,-326144.684020,-360693.300325
All,0.931969,-1380306.167733,-321014.309291,-411895.060681


## Без weekday_6

In [22]:
expanded_df_without_outliers = expand(df_without_outliers)
expanded_df_without_outliers['day'] = range(len(expanded_df_without_outliers))

expanded_df_wo_outliers_wo_weekday_6 = expanded_df_without_outliers.drop(columns=['weekday_6'])
model_wo_weekday, plot_wo_weekday, scores_wo_weekday = fit_model(expanded_df_wo_outliers_wo_weekday_6, 14)

predictor,coef,pvalue
intercept,6939560.5779,0.000000e+00
month_1,202695.5905,4.012483e-02
month_2,433925.4860,2.118173e-05
month_3,-238353.2353,1.500004e-02
month_4,-991229.4902,0.000000e+00
month_5,-2345896.9559,0.000000e+00
month_6,-3551845.6496,0.000000e+00
month_7,-4207452.1652,0.000000e+00
month_8,-3677044.1078,0.000000e+00
month_9,-2755136.4704,0.000000e+00


In [23]:
scores_wo_weekday.style.highlight_max()

,r2,max_error,neg_mean_absolute_error,neg_root_mean_squared_error
Train,0.929018,-1380383.546293,-321518.956179,-415544.004885
Test,0.503784,-649500.703585,-326099.356767,-360291.729893
All,0.931238,-1380383.546293,-321646.696434,-414103.048328


In [24]:
scores_all.style.highlight_max()

,r2,max_error,neg_mean_absolute_error,neg_root_mean_squared_error
Train,0.929792,-1380306.167733,-320867.126409,-413270.369546
Test,0.502677,-649318.640957,-326144.684020,-360693.300325
All,0.931969,-1380306.167733,-321014.309291,-411895.060681


## Без weekday_6 и month_1

In [25]:
expanded_df_without_outliers = expand(df_without_outliers)
expanded_df_without_outliers['day'] = range(len(expanded_df_without_outliers))

expanded_df_wo_outliers_wo_weekday_6_wo_month_1 = expanded_df_without_outliers.drop(columns=['weekday_6', 'month_1'])
model_wo_weekday_wo_month, plot_wo_weekday_wo_month, scores_wo_weekday_wo_month = (
    fit_model(expanded_df_wo_outliers_wo_weekday_6_wo_month_1, 14)
)

predictor,coef,pvalue
intercept,7029681.3401,0.000000e+00
month_2,353176.8398,1.765419e-04
month_3,-317807.7541,4.526054e-04
month_4,-1069497.1447,0.000000e+00
month_5,-2423089.3732,0.000000e+00
month_6,-3627565.8561,0.000000e+00
month_7,-4282056.9482,0.000000e+00
month_8,-3755126.8207,0.000000e+00
month_9,-2834526.3082,0.000000e+00
month_10,-1173085.0696,0.000000e+00


In [26]:
scores_wo_weekday_wo_month.style.highlight_max()

,r2,max_error,neg_mean_absolute_error,neg_root_mean_squared_error
Train,0.928379,-1380074.895294,-321840.639501,-417408.109735
Test,0.506052,-581218.898293,-336699.015090,-359467.491697
All,0.930639,-1380074.895294,-322255.016509,-415901.667627


In [27]:
scores_wo_weekday.style.highlight_max()

,r2,max_error,neg_mean_absolute_error,neg_root_mean_squared_error
Train,0.929018,-1380383.546293,-321518.956179,-415544.004885
Test,0.503784,-649500.703585,-326099.356767,-360291.729893
All,0.931238,-1380383.546293,-321646.696434,-414103.048328


In [28]:
plot_wo_weekday_wo_month

Unsupported

# Результаты

In [29]:
NUMBER_OF_DAYS_TO_PREDICT = 7

In [30]:
future_extension = pd.DataFrame(data=[None] * NUMBER_OF_DAYS_TO_PREDICT,
                                index=pd.date_range(df_without_outliers.index.max() + + timedelta(days=1),
                                                    df_without_outliers.index.max() + timedelta(
                                                        days=NUMBER_OF_DAYS_TO_PREDICT)).date,
                                columns=['value'])
extended_df_without_outliers = pd.concat([df_without_outliers, future_extension])
expanded_and_extended_df_wo_outlires = expand(pd.concat([df_without_outliers, future_extension]))
expanded_and_extended_df_wo_outlires['day'] = range(len(expanded_and_extended_df_wo_outlires))
expanded_and_extended_df_wo_outlires.drop(columns=['month_1', 'weekday_6'], inplace=True)

x_columns = expanded_and_extended_df_wo_outlires.columns.tolist()
x_columns.remove('value')

model = LinearRegression()
model.fit(expanded_and_extended_df_wo_outlires[x_columns][:-NUMBER_OF_DAYS_TO_PREDICT],
          expanded_and_extended_df_wo_outlires['value'][:-NUMBER_OF_DAYS_TO_PREDICT])

fig = go.Figure()

fig.add_scatter(
    x=expanded_and_extended_df_wo_outlires.index,
    y=expanded_and_extended_df_wo_outlires['value'],
    name='Исходные данные',
)

predictions = model.predict(expanded_and_extended_df_wo_outlires[x_columns])

fig.add_scatter(
    x=expanded_and_extended_df_wo_outlires.index,
    y=predictions,
    name='Модель',
)
fig.update_layout(xaxis_title='Дата', yaxis_title='Объем потребления',
                  title=f'Объём потребления электроэнергии в период с 2009 по 2010 год + {NUMBER_OF_DAYS_TO_PREDICT} дней')
fig.show()


Unsupported

In [31]:
score_model(model, expanded_and_extended_df_wo_outlires[x_columns][:-NUMBER_OF_DAYS_TO_PREDICT],
            expanded_and_extended_df_wo_outlires['value'][:-NUMBER_OF_DAYS_TO_PREDICT], None, None)

,r2,max_error,neg_mean_absolute_error,neg_root_mean_squared_error
Train,0.930659,-1.385022e+06,-322047.424834,-415842.61559


In [32]:
pd.DataFrame(data=predictions[-NUMBER_OF_DAYS_TO_PREDICT:],
             index=pd.date_range(df_without_outliers.index.max() + + timedelta(days=1),
                                 df_without_outliers.index.max() + timedelta(
                                     days=NUMBER_OF_DAYS_TO_PREDICT)).date,
             columns=['value'])

,value
2011-01-01,8.020356e+06
2011-01-02,8.022351e+06
2011-01-03,8.397069e+06
2011-01-04,8.532628e+06
2011-01-05,8.563304e+06
2011-01-06,8.539024e+06
2011-01-07,8.483085e+06
